In [2]:

from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2025-06-23'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%***********************]  503 of 503 completed


Price                   close        high         low        open       volume
date       ticker                                                             
2017-06-26 A        55.760605   56.428903   55.675891   56.278300    1535500.0
           AAPL     33.965790   34.538795   33.863300   34.280243  102769600.0
           ABBV     51.661907   51.803956   51.427536   51.697421    4019500.0
           ABT      42.433060   42.901551   42.415708   42.693331    5302600.0
           ACGL     29.446157   29.629997   29.373254   29.427139    1354500.0
...                       ...         ...         ...         ...          ...
2025-06-20 XYL     124.459999  126.139999  124.349998  125.190002    2027300.0
           YUM     139.059998  139.759995  138.559998  138.759995    2956500.0
           ZBH      91.220001   91.489998   90.550003   90.940002    4060800.0
           ZBRA    294.040009  295.739990  290.600006  295.100006     777900.0
           ZTS     157.169998  157.699997  155.000000  155.570007    3296100.0

[990018 rows x 5 columns]

### Calculate features and tech indicators = garman klass vol, rsi, BB, atr macd & $ volume

In [3]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['close'].apply(compute_macd)

df['dollar_vol'] = (df['close']*df['volume'])/1e6

df

Price                   close        high         low        open  \
date       ticker                                                   
2017-06-26 A        55.760605   56.428903   55.675891   56.278300   
           AAPL     33.965790   34.538795   33.863300   34.280243   
           ABBV     51.661907   51.803956   51.427536   51.697421   
           ABT      42.433060   42.901551   42.415708   42.693331   
           ACGL     29.446157   29.629997   29.373254   29.427139   
...                       ...         ...         ...         ...   
2025-06-20 XYL     124.459999  126.139999  124.349998  125.190002   
           YUM     139.059998  139.759995  138.559998  138.759995   
           ZBH      91.220001   91.489998   90.550003   90.940002   
           ZBRA    294.040009  295.739990  290.600006  295.100006   
           ZTS     157.169998  157.699997  155.000000  155.570007   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2017-06-26 A         1535500.0          0.000057        NaN       NaN   
           AAPL    102769600.0          0.000162        NaN       NaN   
           ABBV      4019500.0          0.000026        NaN       NaN   
           ABT       5302600.0          0.000050        NaN       NaN   
           ACGL      1354500.0          0.000038        NaN       NaN   
...                        ...               ...        ...       ...   
2025-06-20 XYL       2027300.0          0.000089  50.553138  4.826650   
           YUM       2956500.0          0.000035  38.170649  4.945038   
           ZBH       4060800.0          0.000050  42.174728  4.509740   
           ZBRA       777900.0          0.000149  54.869536  5.651271   
           ZTS       3296100.0          0.000109  44.299336  5.065233   

Price                bb_mid   bb_high       atr      macd   dollar_vol  
date       ticker                                                       
2017-06-26 A            NaN       NaN       NaN       NaN    85.620409  
           AAPL         NaN       NaN       NaN       NaN  3490.650631  
           ABBV         NaN       NaN       NaN       NaN   207.655036  
           ABT          NaN       NaN       NaN       NaN   225.005542  
           ACGL         NaN       NaN       NaN       NaN    39.884820  
...                     ...       ...       ...       ...          ...  
2025-06-20 XYL     4.845216  4.863782 -0.011728  0.296674   252.317756  
           YUM     4.970724  4.996410  0.794248 -1.219467   411.130883  
           ZBH     4.535581  4.561422 -0.754887 -0.570739   370.426181  
           ZBRA    5.678665  5.706058 -0.228401  0.371306   228.733723  
           ZTS     5.114256  5.163279  0.599101 -0.235623   518.048031  

[990018 rows x 13 columns]

In [4]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_vol', 'volume', 'open',
                                                          'high', 'low', 'ajd close']]

data = (pd.concat([df.unstack('ticker')['dollar_vol'].resample('M').mean().stack('ticker').to_frame('dollar_vol'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1))


In [5]:
data['dollar_vol'] = (data.loc[:, 'dollar_vol'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_vol'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_vol', 'dollar_vol_rank'], axis=1)

data

close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2018-05-31 AAPL     44.204182          0.000061  59.945698  3.779782   
           ABBV     72.653091          0.000186  46.193211  4.288175   
           ABT      54.458668          0.000054  54.184865  3.961810   
           ACN     140.315765          0.000071  53.108613  4.925236   
           ADBE    249.279999          0.000167  67.105815  5.431916   
...                       ...               ...        ...       ...   
2025-06-30 VZ       41.700001          0.000055  40.540867  3.755464   
           WDAY    238.190002          0.000143  41.958813  5.452324   
           WFC      75.400002          0.000043  55.578304  4.292635   
           WMT      96.120003          0.000035  50.706450  4.547913   
           XOM     114.699997          0.000086  65.586701  4.593714   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2018-05-31 AAPL    3.809634  3.839485 -1.272356  0.089921  
           ABBV    4.337685  4.387196 -0.337959  0.028437  
           ABT     4.004612  4.047413 -1.189699  0.210199  
           ACN     4.944963  4.964690 -1.209980 -0.011233  
           ADBE    5.480050  5.528185 -1.223800  0.450243  
...                     ...       ...       ...       ...  
2025-06-30 VZ      3.788644  3.821825  0.017219 -0.912176  
           WDAY    5.514067  5.575810 -0.248535 -0.692321  
           WFC     4.323490  4.354346  1.294210  0.240615  
           WMT     4.583294  4.618676  2.013154 -0.790452  
           XOM     4.676438  4.759162  0.812764  1.600801  

[12814 rows x 8 columns]

In [6]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['close'].pct_change(lag).pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),upper=x.quantile(1-outlier_cutoff))).add(1).pow(1/lag).sub(1))
    return df
    
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns)

data

close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2018-05-31 AAPL     44.204182          0.000061  59.945698  3.779782   
           ABBV     72.653091          0.000186  46.193211  4.288175   
           ABT      54.458668          0.000054  54.184865  3.961810   
           ACN     140.315765          0.000071  53.108613  4.925236   
           ADBE    249.279999          0.000167  67.105815  5.431916   
...                       ...               ...        ...       ...   
2025-06-30 VZ       41.700001          0.000055  40.540867  3.755464   
           WDAY    238.190002          0.000143  41.958813  5.452324   
           WFC      75.400002          0.000043  55.578304  4.292635   
           WMT      96.120003          0.000035  50.706450  4.547913   
           XOM     114.699997          0.000086  65.586701  4.593714   

                     bb_mid   bb_high       atr      macd  return_1m  \
date       ticker                                                      
2018-05-31 AAPL    3.809634  3.839485 -1.272356  0.089921        NaN   
           ABBV    4.337685  4.387196 -0.337959  0.028437        NaN   
           ABT     4.004612  4.047413 -1.189699  0.210199        NaN   
           ACN     4.944963  4.964690 -1.209980 -0.011233        NaN   
           ADBE    5.480050  5.528185 -1.223800  0.450243        NaN   
...                     ...       ...       ...       ...        ...   
2025-06-30 VZ      3.788644  3.821825  0.017219 -0.912176  -0.051410   
           WDAY    5.514067  5.575810 -0.248535 -0.692321  -0.038432   
           WFC     4.323490  4.354346  1.294210  0.240615   0.008291   
           WMT     4.583294  4.618676  2.013154 -0.790452  -0.026337   
           XOM     4.676438  4.759162  0.812764  1.600801   0.121212   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2018-05-31 AAPL          NaN        NaN        NaN        NaN         NaN  
           ABBV          NaN        NaN        NaN        NaN         NaN  
           ABT           NaN        NaN        NaN        NaN         NaN  
           ACN           NaN        NaN        NaN        NaN         NaN  
           ADBE          NaN        NaN        NaN        NaN         NaN  
...                      ...        ...        ...        ...         ...  
2025-06-30 VZ      -0.027150  -0.022475   0.012641  -0.002793    0.006380  
           WDAY    -0.013996   0.006608  -0.013246  -0.002860    0.005296  
           WFC      0.033270   0.018344   0.013640   0.034456    0.022110  
           WMT     -0.004626   0.031508   0.011240   0.020384    0.030504  
           XOM      0.046837  -0.008975   0.013803   0.000503    0.002555  

[12814 rows x 14 columns]

In [7]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-05-31 AAPL    0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
           ABBV    0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
           ABT     0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
           ACN     0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
           ADBE    0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
...                   ...     ...     ...     ...     ...        ...
2025-04-30 VZ     -0.0085 -0.0185 -0.0341 -0.0284 -0.0267  -0.013058
           WDAY   -0.0085 -0.0185 -0.0341 -0.0284 -0.0267   0.049116
           WFC    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267  -0.010865
           WMT    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267   0.107757
           XOM    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267  -0.111830

[12516 rows x 6 columns]

In [8]:
#filtrer les donnees avec moins de 10mois de data
observations = factor_data.groupby(level=1).size()
valid_stocks = observations[observations >= 10]
factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-05-31 AAPL    0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
           ABBV    0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
           ABT     0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
           ACN     0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
           ADBE    0.0263  0.0471 -0.0314 -0.0198 -0.0149        NaN
...                   ...     ...     ...     ...     ...        ...
2025-04-30 VZ     -0.0085 -0.0185 -0.0341 -0.0284 -0.0267  -0.013058
           WDAY   -0.0085 -0.0185 -0.0341 -0.0284 -0.0267   0.049116
           WFC    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267  -0.010865
           WMT    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267   0.107757
           XOM    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267  -0.111830

[12456 rows x 6 columns]

In [9]:
betas =(factor_data.groupby(level=1,
                    group_keys=False)
                    .apply(lambda x: RollingOLS(endog=x['return_1m'], 
                            exog=sm.add_constant(x.drop('return_1m', axis=1)), 
                            window=min(24, x.shape[0]),
                            min_nobs=len(x.columns)+1)
                            .fit(params_only=True)
                            .params
                            .drop('const', axis=1)))
betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2018-05-31 AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
           ADBE         NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2025-04-30 VZ      0.904505 -1.063886  0.905450  0.658103 -0.070860
           WDAY    1.108715 -0.937077  0.037348 -1.896113 -0.421493
           WFC     0.877759 -0.095224  0.559151 -1.655161 -0.085206
           WMT     0.787343  0.091219 -0.722476  0.394638  0.633480
           XOM     0.476822 -0.287362  1.039672  1.215637  0.114549

[12456 rows x 5 columns]

In [12]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

data = pd.concat([data, betas], axis=1)
data.loc[:, factors] = data.groupby('ticker', group_keys=False).apply(lambda x: x.fillna(x.mean()))

data = data.drop('close', axis=1)

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10613 entries, (Timestamp('2019-05-31 00:00:00'), 'AAPL') to (Timestamp('2025-06-30 00:00:00'), 'XOM')
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   garman_klass_vol  10613 non-null  float64
 1   rsi               10613 non-null  float64
 2   bb_low            10613 non-null  float64
 3   bb_mid            10613 non-null  float64
 4   bb_high           10613 non-null  float64
 5   atr               10613 non-null  float64
 6   macd              10613 non-null  float64
 7   return_1m         10613 non-null  float64
 8   return_2m         10613 non-null  float64
 9   return_3m         10613 non-null  float64
 10  return_6m         10613 non-null  float64
 11  return_9m         10613 non-null  float64
 12  return_12m        10613 non-null  float64
 13  Mkt-RF            10613 non-null  float64
 14  SMB               10613 non-null  float64
 15  HML       

In [26]:
#clustering and K-means clustering
from sklearn.cluster import KMeans


def get_clusters(df):
    df['cluster'] = KMeans(n_clusters=4,
                            random_state=0,
                            init=initial_centroids).fit_predict(df)
    return df
data =  data.dropna().groupby('date', group_keys=False).apply(get_clusters)
data

garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2019-05-31 AAPL            0.000128  34.632863  3.729628  3.836748  3.943868   
           ABBV            0.000092  40.509202  4.087179  4.120630  4.154082   
           ABT             0.000071  47.744537  4.223157  4.246252  4.269347   
           ACN             0.000068  54.904696  5.066404  5.093782  5.121161   
           ADBE            0.000117  47.431756  5.595761  5.627233  5.658705   
...                             ...        ...       ...       ...       ...   
2025-06-30 VZ              0.000055  40.540867  3.755464  3.788644  3.821825   
           WDAY            0.000143  41.958813  5.452324  5.514067  5.575810   
           WFC             0.000043  55.578304  4.292635  4.323490  4.354346   
           WMT             0.000035  50.706450  4.547913  4.583294  4.618676   
           XOM             0.000086  65.586701  4.593714  4.676438  4.759162   

                        atr      macd  return_1m  return_2m  return_3m  ...  \
date       ticker                                                       ...   
2019-05-31 AAPL   -1.025160 -0.776558  -0.124213  -0.038120   0.004970  ...   
           ABBV   -1.033814 -0.334901  -0.033758  -0.017923  -0.006405  ...   
           ABT    -0.876499 -0.474087  -0.043112  -0.022127  -0.005087  ...   
           ACN    -1.074949 -0.005044  -0.025182   0.009963   0.036194  ...   
           ADBE   -1.013111 -0.111163  -0.063440   0.008240   0.010555  ...   
...                     ...       ...        ...        ...        ...  ...   
2025-06-30 VZ      0.017219 -0.912176  -0.051410  -0.027150  -0.022475  ...   
           WDAY   -0.248535 -0.692321  -0.038432  -0.013996   0.006608  ...   
           WFC     1.294210  0.240615   0.008291   0.033270   0.018344  ...   
           WMT     2.013154 -0.790452  -0.026337  -0.004626   0.031508  ...   
           XOM     0.812764  1.600801   0.121212   0.046837  -0.008975  ...   

                        SMB       HML       RMW       CMA    Mkt-RF       SMB  \
date       ticker                                                               
2019-05-31 AAPL   -0.172002 -0.654241  0.528277  0.487499  1.272319 -0.172002   
           ABBV    0.362241 -0.042128  0.341551  0.524134  0.454703  0.362241   
           ABT    -0.174475 -0.387485  0.078898  1.035841  0.830895 -0.174475   
           ACN    -0.160495 -0.265179  0.084847  0.047558  1.152980 -0.160495   
           ADBE   -0.505818 -0.223506  0.132702 -0.498643  1.305761 -0.505818   
...                     ...       ...       ...       ...       ...       ...   
2025-06-30 VZ     -0.540765  0.295127  0.267885  0.187318  0.543341 -0.540765   
           WDAY   -0.274654 -0.352603 -0.410693 -0.784819  1.127441 -0.274654   
           WFC    -0.083472  1.463821 -0.893212 -0.797578  1.025696 -0.083472   
           WMT    -0.296028 -0.329861  0.392766  0.582424  0.625368 -0.296028   
           XOM     0.128173  0.719157 -0.142736  0.530061  0.984430  0.128173   

                        HML       RMW       CMA  cluster  
date       ticker                                         
2019-05-31 AAPL   -0.654241  0.528277  0.487499        0  
           ABBV   -0.042128  0.341551  0.524134        3  
           ABT    -0.387485  0.078898  1.035841        3  
           ACN    -0.265179  0.084847  0.047558        2  
           ADBE   -0.223506  0.132702 -0.498643        3  
...                     ...       ...       ...      ...  
2025-06-30 VZ      0.295127  0.267885  0.187318        0  
           WDAY   -0.352603 -0.410693 -0.784819        0  
           WFC     1.463821 -0.893212 -0.797578        3  
           WMT    -0.329861  0.392766  0.582424        3  
           XOM     0.719157 -0.142736  0.530061        1  

[10613 rows x 29 columns]

In [25]:
target_rsi_values = [30,45,55,70]
initial_centroids   = np.zeros((len(target_rsi_values), 28))

initial_centroids 

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
#select the stock we ae giong to invest to in the next: mont:h
#efficient frontiers max sharpe ratio

#data[data['cluster'] == 3].copy()
#filtered_df = data[data['cluster'] == 3].copy()
#filtered_df = filtered_df.reset_index(level=1)
filtered_df

,ticker,garman_klass_vol,rsi,bb_low,bb_mid,bb_high,atr,macd,return_1m,return_2m,...,SMB,HML,RMW,CMA,Mkt-RF,SMB,HML,RMW,CMA,cluster
date,,,,,,,,,,,,,,,,,,,,,
2019-05-31,ABBV,0.000092,40.509202,4.087179,4.120630,4.154082,-1.033814,-0.334901,-0.033758,-0.017923,...,0.362241,-0.042128,0.341551,0.524134,0.454703,0.362241,-0.042128,0.341551,0.524134,3
2019-05-31,ABT,0.000071,47.744537,4.223157,4.246252,4.269347,-0.876499,-0.474087,-0.043112,-0.022127,...,-0.174475,-0.387485,0.078898,1.035841,0.830895,-0.174475,-0.387485,0.078898,1.035841,3
2019-05-31,ADBE,0.000117,47.431756,5.595761,5.627233,5.658705,-1.013111,-0.111163,-0.063440,0.008240,...,-0.505818,-0.223506,0.132702,-0.498643,1.305761,-0.505818,-0.223506,0.132702,-0.498643,3
2019-05-31,ADSK,0.000126,42.624169,5.075266,5.139509,5.203752,-0.662057,-0.571718,-0.097076,0.016202,...,-0.036180,0.314272,-0.771156,-0.906069,1.359588,-0.036180,0.314272,-0.771156,-0.906069,3
2019-05-31,ALGN,0.001627,40.691122,5.679790,5.758807,5.837824,-0.148246,-0.149249,-0.124215,0.000035,...,1.789867,-0.281305,0.863651,-1.594036,0.823186,1.789867,-0.281305,0.863651,-1.594036,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,UBER,0.000189,49.799498,4.416834,4.458825,4.500815,1.122147,-0.108823,-0.004515,0.016953,...,0.308911,-0.006408,-1.698099,-0.489786,1.350648,0.308911,-0.006408,-1.698099,-0.489786,3
2025-06-30,UNP,0.000067,49.581526,5.391664,5.409299,5.426934,-0.197237,-0.049739,0.003835,0.018830,...,-0.111439,0.081512,-0.189316,0.031193,1.035997,-0.111439,0.081512,-0.189316,0.031193,3
2025-06-30,UPS,0.000090,50.271881,4.567267,4.602248,4.637229,-0.811764,0.236059,0.017736,0.028978,...,-0.436268,-0.174900,-0.335956,0.605779,1.200556,-0.436268,-0.174900,-0.335956,0.605779,3
